In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.layers import Dense, Input
from tensorflow.keras.models import Model

# Create synthetic time series data
np.random.seed(42)
time = np.arange(0, 1000, 1)
demand = 0.05 * time + np.sin(time * 0.1) + np.random.normal(0, 0.5, size=len(time))

# Convert to a supervised learning problem
X = np.array([demand[i-20:i] for i in range(20, len(demand))])
Y = np.array([demand[i] for i in range(20, len(demand))])

# Split into train and test sets
split_time = 800
X_train, X_test = X[:split_time], X[split_time:]
Y_train, Y_test = Y[:split_time], Y[split_time:]

# Quantile loss function
def quantile_loss(tau, y_true, y_pred):
    err = y_true - y_pred
    return tf.reduce_mean(tf.maximum(tau * err, (tau - 1) * err), axis=-1)

# Building the model to predict three quantiles
input_layer = Input(shape=(20,))
hidden_layer = Dense(128, activation='relu')(input_layer)
hidden_layer_2 = Dense(128, activation='relu')(hidden_layer)
output_layer = Dense(3, activation='linear')(hidden_layer_2)

model = Model(inputs=input_layer, outputs=output_layer)

# Compile the model
model.compile(optimizer='adam', loss=lambda y_true, y_pred: quantile_loss(0.5, y_true[:, 0], y_pred[:, 0]) +
                                                           quantile_loss(0.25, y_true[:, 1], y_pred[:, 1]) +
                                                           quantile_loss(0.75, y_true[:, 2], y_pred[:, 2]))

# Adjust Y for training (include three quantiles)
Y_train_q = np.tile(Y_train, (3, 1)).T  # Repeat Y_train 3 times and transpose
Y_test_q = np.tile(Y_test, (3, 1)).T  # Repeat Y_test 3 times and transpose

# Train the model
model.fit(X_train, Y_train_q, epochs=100, batch_size=16, verbose=0)

# Predict using the trained model (example)
predictions = model.predict(X_test[:10])
print(f"Predictions for the first 10 test samples:\n{predictions}")


: 